In [1]:
import os
import base64
from dotenv import load_dotenv
from requests import post
import json
load_dotenv()

True

In [2]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
print(client_id, client_secret)

8ed09b2a76e443b0bd22e79203485ed9 c08b968943454f19a875b4ae7c38aa5d


In [3]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

# Function to create the authorization header
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

# Function to search for an artist by name
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"q={artist_name}&type=artist&limit=1"
    
    query_url = f"{url}?{query}"
    result = requests.get(query_url, headers=headers)
    
    return json.loads(result.content)

# Function to dump artist data into a CSV
def dump_artist_to_csv(artist_data):
    artist_info = {
        'name': [artist_data['name']],
        'popularity': [artist_data['popularity']],
        'followers': [artist_data['followers']['total']],
        'genres': [", ".join(artist_data['genres'])],
        'spotify_url': [artist_data['external_urls']['spotify']]
    }

    df = pd.DataFrame(artist_info)
    df.to_csv('burna_boy_data.csv', index=False)
    print("Data saved to burna_boy_data.csv")


# Main flow
token = get_token()

# Search for Burna Boy
artist_name = "Burna Boy"
result = search_for_artist(token, artist_name)

# Extract the artist's data
if result['artists']['items']:
    burna_boy_data = result['artists']['items'][0]
    dump_artist_to_csv(burna_boy_data)
else:
    print("Artist not found")

    

NameError: name 'requests' is not defined

In [34]:
def get_artist_albums(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    
    params = {
        "include_groups": "album",  # We are only interested in albums
        "limit": 50  # Maximum limit per request
    }
    
    result = requests.get(url, headers=headers, params=params)
    return json.loads(result.content)

# Function to dump album data into a CSV
def dump_albums_to_csv(albums):
    albums_data = []
    
    for album in albums['items']:
        albums_data.append({
            'album_name': album['name'],
            'release_date': album['release_date'],
            'total_tracks': album['total_tracks'],
            'album_type': album['album_type'],
            'spotify_url': album['external_urls']['spotify']
        })
    
    df = pd.DataFrame(albums_data)
    df.to_csv('burna_boy_albums.csv', index=False)
    print("Albums data saved to burna_boy_albums.csv")

# Main flow
token = get_token()

# Search for Burna Boy
artist_name = "Burna Boy"
result = search_for_artist(token, artist_name)

# Extract the artist's data
if result['artists']['items']:
    burna_boy_data = result['artists']['items'][0]
    artist_id = burna_boy_data['id']  # Get Burna Boy's artist ID
    
    # Fetch the albums of Burna Boy
    albums = get_artist_albums(token, artist_id)
    
    # Dump the albums to CSV
    dump_albums_to_csv(albums)
else:
    print("Artist not found")

Albums data saved to burna_boy_albums.csv


In [38]:

# Function to get detailed information about an album, including tracks
def get_album_tracks(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = get_auth_header(token)
    
    result = requests.get(url, headers=headers)
    return json.loads(result.content)

# Function to get detailed information about a track
def get_track_details(token, track_id):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = get_auth_header(token)
    
    result = requests.get(url, headers=headers)
    return json.loads(result.content)

# Function to gather and save track statistics
def get_track_stats_for_albums(albums, token):
    track_data = []

    for album in albums['items']:
        # Get all tracks for the album
        album_tracks = get_album_tracks(token, album['id'])
        album_name = album['name']
        release_date = album['release_date']
        
        for track in album_tracks['items']:
            # Get detailed information about each track
            track_details = get_track_details(token, track['id'])
            
            # Collect track data
            track_data.append({
                'album_name': album_name,
                'track_name': track_details['name'],
                'track_number': track_details['track_number'],
                'popularity': track_details['popularity'],  # Popularity score as a proxy for total listens
                'duration_min': track_details['duration_ms'] / 60000,  # Convert milliseconds to minutes
                'explicit': track_details['explicit'],  # True if the track has explicit content
                'release_date': release_date
            })

    # Convert the track data to a DataFrame
    df = pd.DataFrame(track_data)

    # Save to CSV for easy access
    df.to_csv('burna_boy_tracks_stats.csv', index=False)
    print("Track stats saved to burna_boy_tracks_stats.csv")

# Main flow
token = get_token()

# Search for Burna Boy
artist_name = "Burna Boy"
result = search_for_artist(token, artist_name)

# Extract the artist's data
if result['artists']['items']:
    burna_boy_data = result['artists']['items'][0]
    artist_id = burna_boy_data['id']  # Get Burna Boy's artist ID
    
    # Fetch the albums of Burna Boy
    albums = get_artist_albums(token, artist_id)
    
    # Get track stats and save them to CSV
    get_track_stats_for_albums(albums, token)
else:
    print("Artist not found")

Track stats saved to burna_boy_tracks_stats.csv
